## EDA - Decision Tree for Titanic

**Import Package**

In [213]:
import pandas as pd

**Load Dataset**

In [214]:
train = pd.read_csv('input/train.csv', index_col = "PassengerId")    # train Dataset load
print(train.shape)
train.head()

(891, 11)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [215]:
test = pd.read_csv('input/test.csv', index_col = "PassengerId")    # test Dataset load
print(test.shape)
test.head()

(418, 10)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [216]:
submit = pd.read_csv('input/gender_submission.csv', index_col = "PassengerId")    # submission template load
print(submit.shape)
submit.head()

(418, 1)


,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


## Preprocess
**Encode Sex**

In [217]:
train['Sex_Encoded'] = train['Sex'].replace('male', 0)\
                                   .replace('female', 1)

test['Sex_Encoded'] = test['Sex'].replace('male', 0)\
                                 .replace('female', 1)

In [218]:
# Check Encode Datd - Train
train[['Sex', 'Sex_Encoded']].head()

,Sex,Sex_Encoded
PassengerId,,
1,male,0
2,female,1
3,female,1
4,female,1
5,male,0


In [219]:
# Check Encode Data - Test
test[['Sex', 'Sex_Encoded']].head()

,Sex,Sex_Encoded
PassengerId,,
892,male,0
893,female,1
894,male,0
895,male,0
896,female,1


**Fill Missing Fare - Test -**

In [220]:
# Search Missing Fare - train
train.isnull().sum()

Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Sex_Encoded      0
dtype: int64

In [221]:
test[test['Fare'].isnull()]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_Encoded
PassengerId,,,,,,,,,,,
1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S,0


In [222]:
# Search Missing Fare - test
test.isnull().sum()
# test.csv missing fare 1 fill 0
test['Fare'] = test['Fare'].fillna(0)
# Check missing fare
test.isnull().sum()

Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
Sex_Encoded      0
dtype: int64

**Encode Embarked**

In [223]:
# One Hot Encoding - over 2 data
# C == [True, False, False]
# S == [False, True, False]
# Q == [False, False, True]

# train Embarked encode
train['Embarked_C'] = train['Embarked'] == 'C'
train['Embarked_S'] = train['Embarked'] == 'S'
train['Embarked_Q'] = train['Embarked'] == 'Q'

print(train.shape)
train[['Embarked', 'Embarked_C', 'Embarked_S', 'Embarked_Q']].head()

(891, 15)


,Embarked,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,
1,S,False,True,False
2,C,True,False,False
3,S,False,True,False
4,S,False,True,False
5,S,False,True,False


In [224]:
# test Embarked encode
test['Embarked_C'] = test['Embarked'] == 'C'
test['Embarked_S'] = test['Embarked'] == 'S'
test['Embarked_Q'] = test['Embarked'] == 'Q'

print(test.shape)
test[['Embarked', 'Embarked_C', 'Embarked_S', 'Embarked_Q']].head()

(418, 14)


,Embarked,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,
892,Q,False,False,True
893,S,False,True,False
894,Q,False,False,True
895,S,False,True,False
896,S,False,True,False


**Age Encode Pre-Process**

In [225]:
# Check Age missing-data - train
train[train['Age'].isnull()]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_Encoded,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,,,,,,,,,,,,
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,0,False,False,True
18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S,0,False,True,False
20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C,1,True,False,False
27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C,0,True,False,False
29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q,1,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,0,3,"Razi, Mr. Raihed",male,NaN,0,0,2629,7.2292,NaN,C,0,True,False,False
864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,S,1,False,True,False
869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5000,NaN,S,0,False,True,False


In [226]:
# Age Fill temp data
mean_age_by_pclass = train.groupby('Pclass')['Age'].mean()
mean_age_by_pclass

Pclass
1    38.233441
2    29.877630
3    25.140620
Name: Age, dtype: float64

In [227]:
train['Age(Fill)'] = train['Age']
print(train.shape)
train[['Pclass', 'Age', 'Age(Fill)']].head(30)

(891, 16)


,Pclass,Age,Age(Fill)
PassengerId,,,
1,3,22.0,22.0
2,1,38.0,38.0
3,3,26.0,26.0
4,1,35.0,35.0
5,3,35.0,35.0
6,3,NaN,NaN
7,1,54.0,54.0
8,3,2.0,2.0
9,3,27.0,27.0


In [228]:
train.loc[(train['Age'].isnull()) & (train['Pclass'] == 1), 'Age(Fill)'] = mean_age_by_pclass.loc[1]
train.loc[(train['Age'].isnull()) & (train['Pclass'] == 2), 'Age(Fill)'] = mean_age_by_pclass.loc[2]
train.loc[(train['Age'].isnull()) & (train['Pclass'] == 3), 'Age(Fill)'] = mean_age_by_pclass.loc[3]

In [229]:
print(train.shape)
train[['Pclass', 'Age', 'Age(Fill)']].head(30)

(891, 16)


,Pclass,Age,Age(Fill)
PassengerId,,,
1,3,22.0,22.00000
2,1,38.0,38.00000
3,3,26.0,26.00000
4,1,35.0,35.00000
5,3,35.0,35.00000
6,3,NaN,25.14062
7,1,54.0,54.00000
8,3,2.0,2.00000
9,3,27.0,27.00000


In [230]:
# AgeType - Train
train.loc[train['Age(Fill)']<15,'AgeType']="Young"
train.loc[(train['Age(Fill)']>=15)&(train['Age(Fill)']<30),'AgeType']="Medium"
train.loc[(train['Age(Fill)']>=30),'AgeType']="Old"
train[['Age', 'Age(Fill)', 'AgeType']].head(30)

,Age,Age(Fill),AgeType
PassengerId,,,
1,22.0,22.00000,Medium
2,38.0,38.00000,Old
3,26.0,26.00000,Medium
4,35.0,35.00000,Old
5,35.0,35.00000,Old
6,NaN,25.14062,Medium
7,54.0,54.00000,Old
8,2.0,2.00000,Young
9,27.0,27.00000,Medium


In [231]:
# Check Age missing-data - test
test[test['Age'].isnull()]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_Encoded,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,,,,,,,,,,,
902,3,"Ilieff, Mr. Ylio",male,NaN,0,0,349220,7.8958,NaN,S,0,False,True,False
914,1,"Flegenheim, Mrs. Alfred (Antoinette)",female,NaN,0,0,PC 17598,31.6833,NaN,S,1,False,True,False
921,3,"Samaan, Mr. Elias",male,NaN,2,0,2662,21.6792,NaN,C,0,True,False,False
925,3,"Johnston, Mrs. Andrew G (Elizabeth Lily"" Watson)""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,1,False,True,False
928,3,"Roth, Miss. Sarah A",female,NaN,0,0,342712,8.0500,NaN,S,1,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300,3,"Riordan, Miss. Johanna Hannah""""",female,NaN,0,0,334915,7.7208,NaN,Q,1,False,False,True
1302,3,"Naughton, Miss. Hannah",female,NaN,0,0,365237,7.7500,NaN,Q,1,False,False,True
1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,0,False,True,False


In [232]:
# test mean age
mean_age_by_pclass_test = test.groupby('Pclass')['Age'].mean()
mean_age_by_pclass_test

Pclass
1    40.918367
2    28.777500
3    24.027945
Name: Age, dtype: float64

In [233]:
test['Age(Fill)'] = test['Age']
print(test.shape)
test[['Pclass', 'Age', 'Age(Fill)']].head(30)

(418, 15)


,Pclass,Age,Age(Fill)
PassengerId,,,
892,3,34.5,34.5
893,3,47.0,47.0
894,2,62.0,62.0
895,3,27.0,27.0
896,3,22.0,22.0
897,3,14.0,14.0
898,3,30.0,30.0
899,2,26.0,26.0
900,3,18.0,18.0


In [234]:
test.loc[(test['Age'].isnull()) & (test['Pclass'] == 1), 'Age(Fill)'] = mean_age_by_pclass_test.loc[1]
test.loc[(test['Age'].isnull()) & (test['Pclass'] == 2), 'Age(Fill)'] = mean_age_by_pclass_test.loc[2]
test.loc[(test['Age'].isnull()) & (test['Pclass'] == 3), 'Age(Fill)'] = mean_age_by_pclass_test.loc[3]
test[['Pclass', 'Age', 'Age(Fill)']].head(30)

,Pclass,Age,Age(Fill)
PassengerId,,,
892,3,34.5,34.500000
893,3,47.0,47.000000
894,2,62.0,62.000000
895,3,27.0,27.000000
896,3,22.0,22.000000
897,3,14.0,14.000000
898,3,30.0,30.000000
899,2,26.0,26.000000
900,3,18.0,18.000000


In [235]:
# AgeType - Test
test.loc[test['Age(Fill)']<15,'AgeType']="Young"
test.loc[(test['Age(Fill)']>=15)&(test['Age(Fill)']<30),'AgeType']="Medium"
test.loc[(test['Age(Fill)']>=30),'AgeType']="Old"
test[['Age', 'Age(Fill)', 'AgeType']].head(30)

,Age,Age(Fill),AgeType
PassengerId,,,
892,34.5,34.500000,Old
893,47.0,47.000000,Old
894,62.0,62.000000,Old
895,27.0,27.000000,Medium
896,22.0,22.000000,Medium
897,14.0,14.000000,Young
898,30.0,30.000000,Old
899,26.0,26.000000,Medium
900,18.0,18.000000,Medium


**Age Encode**

In [236]:
# One Hot Encoding - over 2 data
# Young == [True, False, False]
# Medium == [False, True, False]
# Old == [False, False, True]
train['Age_Y'] = train['AgeType'] == 'Young'
train['Age_M'] = train['AgeType'] == 'Medium'
train['Age_O'] = train['AgeType'] == 'Old'

print(train.shape)
train[['Age(Fill)', 'Age_M', 'Age_O', 'Age_Y']].head()

(891, 20)


,Age(Fill),Age_M,Age_O,Age_Y
PassengerId,,,,
1,22.0,True,False,False
2,38.0,False,True,False
3,26.0,True,False,False
4,35.0,False,True,False
5,35.0,False,True,False


In [237]:
# test Age encode
test['Age_Y'] = test['AgeType'] == 'Young'
test['Age_M'] = test['AgeType'] == 'Medium'
test['Age_O'] = test['AgeType'] == 'Old'

print(test.shape)
test[['Age(Fill)', 'Age_M', 'Age_O', 'Age_Y']].head()

(418, 19)


,Age(Fill),Age_M,Age_O,Age_Y
PassengerId,,,,
892,34.5,False,True,False
893,47.0,False,True,False
894,62.0,False,True,False
895,27.0,True,False,False
896,22.0,True,False,False


## Decision Tree Training

**Feature setup**

In [238]:
# feature var-set
feature_names = ['Pclass', 'Sex_Encoded', 'Fare','Embarked_C', 'Embarked_S', 'Embarked_Q', 'Age_M', 'Age_O', 'Age_Y']
feature_names

['Pclass',
 'Sex_Encoded',
 'Fare',
 'Embarked_C',
 'Embarked_S',
 'Embarked_Q',
 'Age_M',
 'Age_O',
 'Age_Y']

In [239]:
# x-axis train feature
x_train = train[feature_names]
print(x_train.shape)
x_train.head()

(891, 9)


,Pclass,Sex_Encoded,Fare,Embarked_C,Embarked_S,Embarked_Q,Age_M,Age_O,Age_Y
PassengerId,,,,,,,,,
1,3,0,7.2500,False,True,False,True,False,False
2,1,1,71.2833,True,False,False,False,True,False
3,3,1,7.9250,False,True,False,True,False,False
4,1,1,53.1000,False,True,False,False,True,False
5,3,0,8.0500,False,True,False,False,True,False


In [240]:
# x-axis test feature
x_test = test[feature_names]
print(x_test.shape)
x_test.head()

(418, 9)


,Pclass,Sex_Encoded,Fare,Embarked_C,Embarked_S,Embarked_Q,Age_M,Age_O,Age_Y
PassengerId,,,,,,,,,
892,3,0,7.8292,False,False,True,False,True,False
893,3,1,7.0000,False,True,False,False,True,False
894,2,0,9.6875,False,False,True,False,True,False
895,3,0,8.6625,False,True,False,True,False,False
896,3,1,12.2875,False,True,False,True,False,False


**Label setup**

In [241]:
label_name = 'Survived'

y_train = train[label_name]
print(y_train.shape)
y_train.head()

(891,)


PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

## Use Decision Tree
**import sklearn**

In [242]:
from sklearn.tree import DecisionTreeClassifier

# max depth limit
model = DecisionTreeClassifier(max_depth = 5)

**Fit & Predict**

In [243]:
# Fitting - Train
model.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [244]:
# Predict - Test
prediction = model.predict(x_test)
print(prediction.shape)
prediction[0:5]

(418,)


array([0, 0, 0, 0, 1], dtype=int64)

## Submission
**Input Submit Data**

In [245]:
submit['Survived'] = prediction

print(submit.shape)
submit.head()

(418, 1)


,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


**Make Submit File**

In [246]:
submit.to_csv('input/decisiontree03-1.csv')